# 신경망 모델

## import libraries

In [2]:
import numpy as np
np.random.seed()
import pandas as pd
from datetime import datetime
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler

from keras.models import load_model
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, Callback
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


## ComputeGini callback
- ROC_AUC계산 후 gini를 구하고
- parameter의 log를 생성하여 early stopping에 사용

In [4]:
class roc_auc_callback(Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict_proba(self.x, verbose=0)
        roc = roc_auc_score(self.y, y_pred)
        logs['roc_auc'] = roc_auc_score(self.y, y_pred)
        logs['norm_gini'] = ( roc_auc_score(self.y, y_pred) * 2 ) - 1

        y_pred_val = self.model.predict_proba(self.x_val, verbose=0)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        logs['roc_auc_val'] = roc_auc_score(self.y_val, y_pred_val)
        logs['norm_gini_val'] = ( roc_auc_score(self.y_val, y_pred_val) * 2 ) - 1

        print('\rroc_auc: %s - roc_auc_val: %s - norm_gini: %s - norm_gini_val: %s' % (str(round(roc,5)),str(round(roc_val,5)),str(round((roc*2-1),5)),str(round((roc_val*2-1),5))), end=10*' '+'\n')
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return


## Housekeeping utilities
- Timer
- data scale

In [5]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod(
            (datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' %
              (thour, tmin, round(tsec, 2)))

def scale_data(X, scaler=None):
    if not scaler:
        scaler = MinMaxScaler(feature_range=(-1, 1))
        scaler.fit(X)
    X = scaler.transform(X)
    return X, scaler

## Load dataset

In [6]:
# train and test data path
DATA_TRAIN_PATH = './porto/train.csv'
DATA_TEST_PATH = './porto/test.csv'

def load_data(path_train=DATA_TRAIN_PATH, path_test=DATA_TEST_PATH):
    train_loader = pd.read_csv(path_train, dtype={'target': np.int8, 'id': np.int32})
    train = train_loader.drop(['target', 'id'], axis=1)
    train_labels = train_loader['target'].values
    train_ids = train_loader['id'].values
    print('\n Shape of raw train data:', train.shape)

    test_loader = pd.read_csv(path_test, dtype={'id': np.int32})
    test = test_loader.drop(['id'], axis=1)
    test_ids = test_loader['id'].values
    print(' Shape of raw test data:', test.shape)

    return train, train_labels, test, train_ids, test_ids

You can ignore most of the parameters below other than the top two. Obviously, more folds means longer running time, but I can tell you from experience that 10 folds with Keras will usually do better than 4. The number of "runs" should be in the 3-5 range. At a minimum, I suggest 5 folds and 3 independent runs per fold (which will eventually get averaged).  This is because of stochastic nature of neural networks, so one run per fold may or may not produce the best possible result.

**If you can afford it, 10 folds and 5 runs per fold would be my recommendation. Be warned that it may take a day or two, even if you have a GPU.**

## Cross Validation
![](./porto/k_fold.png)
- 10 fold cross validation 사용
- 각 fold 마다 5번의 run을 한다, 신경망은 확률적인 특성을 갖기 때문. 

In [7]:
folds = 4
runs = 2

cv_LL = 0
cv_AUC = 0
cv_gini = 0
fpred = []
avpred = []
avreal = []
avids = []

Loading data. Converting "categorical" variables, even though in this dataset they are actually numeric.

## Dummify categorical data

In [8]:
# Load data set and target values
train, target, test, tr_ids, te_ids = load_data()
n_train = train.shape[0]
train_test = pd.concat((train, test)).reset_index(drop=True)
col_to_drop = train.columns[train.columns.str.endswith('_cat')]
col_to_dummify = train.columns[train.columns.str.endswith('_cat')].astype(str).tolist()

for col in col_to_dummify:
    dummy = pd.get_dummies(train_test[col].astype('category'))
    columns = dummy.columns.astype(str).tolist()
    columns = [col + '_' + w for w in columns]
    dummy.columns = columns
    train_test = pd.concat((train_test, dummy), axis=1)

train_test.drop(col_to_dummify, axis=1, inplace=True)
# 데이터 스케일
train_test_scaled, scaler = scale_data(train_test)
train = train_test_scaled[:n_train, :]
test = train_test_scaled[n_train:, :]
print('\n Shape of processed train data:', train.shape)
print(' Shape of processed test data:', test.shape)


 Shape of raw train data: (595212, 57)
 Shape of raw test data: (892816, 57)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)



 Shape of processed train data: (595212, 227)
 Shape of processed test data: (892816, 227)


## Early Stopping
https://kevinthegrey.tistory.com/110
![](./porto/early_stopping.png)

The two parameters below are worth playing with. Larger patience gives the network a better chance to find solutions when it gets close to the local/global minimum. It also means longer training times. Batch size is one of those parameters that can always be optimized for any given dataset. If you have a GPU, larger batch sizes translate to faster training, but that may or may not be better for the quality of training.

- patience : 개선이 없다고 바로 종료하지 않고 개선이 없는 에포크를 얼마나 기다려 줄 것인 가를 지정합니다. 만약 10이라고 지정하면 개선이 없는 에포크가 10번째 지속될 경우 학습일 종료합니다.

In [9]:
patience = 10
batchsize = 128

## 신경망 구현 순서
Keras 를 사용하면 다음과 같은 순서로 신경망을 구성할 수 있다.

1. Sequential 모형 클래스 객체 생성
2. add 메서드로 레이어 추가.
    - 입력단부터 순차적으로 추가한다.
    - 레이어는 출력 뉴런 갯수를 첫번째 인수로 받는다.
    - 최초의 레이어는 input_dim 인수로 입력 크기를 설정해야 한다.
    - activation 인수로 활성화함수 설정
3. compile 메서드로 모형 완성.
    - loss인수로 비용함수 설정
    - optimizer 인수로 최적화 알고리즘 설정
    - metrics 인수로 트레이닝 단계에서 기록할 성능 기준 설정
4. fit 메서드로 트레이닝
    - nb_epoch 로 에포크(epoch) 횟수 설정
    - batch_size 로 배치크기(batch size) 설정
    - verbose는 학습 중 출력되는 문구를 설정하는 것으로, 주피터노트북(Jupyter Notebook)을 사용할 때는 verbose=2로 설정하여 진행 막대(progress bar)가 나오지 않도록 설정한다.

In [16]:
# Let's split the data into folds. I always use the same random number for reproducibility, 
# and suggest that you do the same (you certainly don't have to use 1001).
# 지정한 fold 수(=10)개 로 split 한다

skf = StratifiedKFold(n_splits=folds, random_state=1001)
starttime = timer(None)
for i, (train_index, test_index) in enumerate(skf.split(train, target)):
    start_time = timer(None)
    X_train, X_val = train[train_index], train[test_index]
    y_train, y_val = target[train_index], target[test_index]
    train_ids, val_ids = tr_ids[train_index], tr_ids[test_index]

    # This definition must be within the for loop or else it will continue training previous model
    # fold 마다 정의해주어야 함. 안그러면 이전 fold 학습한다.
    def baseline_model():
        model = Sequential()
        model.add(
            Dense(
                200, #뉴런 수
                input_dim=X_train.shape[1], #입력 크기, 최초의 레이어는 입력해주어야 함
                kernel_initializer='glorot_normal', ## Xavier initialization, weight를 초기화한다.
                ))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(100, kernel_initializer='glorot_normal'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.25))
        model.add(Dense(50, kernel_initializer='glorot_normal'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.15))
        model.add(Dense(25, kernel_initializer='glorot_normal'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.1))
        model.add(Dense(1, activation='sigmoid'))

        # Compile model
        model.compile(optimizer='adam', metrics = ['accuracy'], loss='binary_crossentropy')

        return model

# This is where we repeat the runs for each fold. If you choose runs=1 above, it will run a 
# regular N-fold procedure.
# 각 fold마다 학습을 반복한다. runs= 1 이면 일반적인 k fold
#########
# It is important to leave the call to random seed here, so each run starts with a different seed.
#########

    for run in range(runs):
        print('\n Fold %d - Run %d\n' % ((i + 1), (run + 1)))
        np.random.seed()

# Lots to unpack here.

# The first callback prints out roc_auc and gini values at the end of each epoch. It must be listed 
# before the EarlyStopping callback, which monitors gini values saved in the previous callback. Make 
# sure to set the mode to "max" because the default value ("auto") will not handle gini properly 
# (it will act as if the model is not improving even when roc/gini go up).

# CSVLogger creates a record of all iterations. Not really needed but it doesn't hurt to have it.

# ModelCheckpoint saves a model each time gini improves. Its mode also must be set to "max" for reasons 
# explained above.

        callbacks = [
            # 위에서 정의했던 콜백 함수
            roc_auc_callback(training_data=(X_train, y_train),validation_data=(X_val, y_val)),  # call this before EarlyStopping
            # 지니계수 earlystopping, mode=max로 설정해야 최대 지니계수를 구한다.
            EarlyStopping(monitor='norm_gini_val', patience=patience, mode='max', verbose=1),
            CSVLogger('keras-5fold-run-01-v1-epochs.log', separator=',', append=False),
            ModelCheckpoint(
                    'keras-5fold-run-01-v1-fold-' + str('%02d' % (i + 1)) + '-run-' + str('%02d' % (run + 1)) + '.check',
                    monitor='norm_gini_val', mode='max', # mode must be set to max or Keras will be confused
                    save_best_only=True,
                    verbose=1)
        ]

# The classifier is defined here. Epochs should be be set to a very large number (not 3 like below) which 
# will never be reached anyway because of early stopping. I usually put 5000 there. Because why not.

        nnet = KerasClassifier(
            # 위에 정의했던 신경망 모델
            build_fn=baseline_model,
# Epoch needs to be set to a very large number ; early stopping will prevent it from reaching
            #어짜피 earlystopping 하니까 에폭은 그냥 큰 수 넣어라
            #콜백 함수는 에폭마다 매번 호출된다.
            epochs=5000,
#             epochs=3,
            batch_size=batchsize,
            validation_data=(X_val, y_val),
            verbose=2,
            shuffle=True,
            callbacks=callbacks)

        fit = nnet.fit(X_train, y_train)
        
# We want the best saved model - not the last one where the training stopped. So we delete the old 
# model instance and load the model from the last saved checkpoint. Next we predict values both for 
# validation and test data, and create a summary of parameters for each run.

        # 한 run이 끝나면 해당 run에서 가장 성능 좋은 모델 불러온 후 predict
        del nnet
        nnet = load_model('keras-5fold-run-01-v1-fold-' + str('%02d' % (i + 1)) + '-run-' + str('%02d' % (run + 1)) + '.check')
        scores_val_run = nnet.predict_proba(X_val, verbose=0)
        LL_run = log_loss(y_val, scores_val_run)
        print('\n Fold %d Run %d Log-loss: %.5f' % ((i + 1), (run + 1), LL_run))
        AUC_run = roc_auc_score(y_val, scores_val_run)
        print(' Fold %d Run %d AUC: %.5f' % ((i + 1), (run + 1), AUC_run))
        print(' Fold %d Run %d normalized gini: %.5f' % ((i + 1), (run + 1), AUC_run*2-1))
        y_pred_run = nnet.predict_proba(test, verbose=0)
        if run > 0:
            scores_val = scores_val + scores_val_run
            y_pred = y_pred + y_pred_run
        else:
            scores_val = scores_val_run
            y_pred = y_pred_run
            
# We average all runs from the same fold and provide a parameter summary for each fold. Unless something 
# is wrong, the numbers printed here should be better than any of the individual runs.

    # 한 fold 끝나고 fold의 run값 평균
    scores_val = scores_val / runs
    y_pred = y_pred / runs
    LL = log_loss(y_val, scores_val)
    print('\n Fold %d Log-loss: %.5f' % ((i + 1), LL))
    AUC = roc_auc_score(y_val, scores_val)
    print(' Fold %d AUC: %.5f' % ((i + 1), AUC))
    print(' Fold %d normalized gini: %.5f' % ((i + 1), AUC*2-1))
    timer(start_time)
    
# We add up predictions on the test data for each fold. Create out-of-fold predictions for validation data.

    if i > 0:
        fpred = pred + y_pred
        avreal = np.concatenate((avreal, y_val), axis=0)
        avpred = np.concatenate((avpred, scores_val), axis=0)
        avids = np.concatenate((avids, val_ids), axis=0)
    else:
        fpred = y_pred
        avreal = y_val
        avpred = scores_val
        avids = val_ids
    pred = fpred
    cv_LL = cv_LL + LL
    cv_AUC = cv_AUC + AUC
    cv_gini = cv_gini + (AUC*2-1)



 Fold 1 - Run 1

Train on 446408 samples, validate on 148804 samples
Epoch 1/5000
 - 41s - loss: 0.1843 - acc: 0.9521 - val_loss: 0.1533 - val_acc: 0.9635
roc_auc: 0.62553 - roc_auc_val: 0.62007 - norm_gini: 0.25106 - norm_gini_val: 0.24014          

Epoch 00001: norm_gini_val improved from -inf to 0.24014, saving model to keras-5fold-run-01-v1-fold-01-run-01.check
Epoch 2/5000
 - 38s - loss: 0.1548 - acc: 0.9635 - val_loss: 0.1530 - val_acc: 0.9635
roc_auc: 0.63092 - roc_auc_val: 0.6249 - norm_gini: 0.26184 - norm_gini_val: 0.24981          

Epoch 00002: norm_gini_val improved from 0.24014 to 0.24981, saving model to keras-5fold-run-01-v1-fold-01-run-01.check
Epoch 3/5000
 - 40s - loss: 0.1539 - acc: 0.9636 - val_loss: 0.1526 - val_acc: 0.9635
roc_auc: 0.63478 - roc_auc_val: 0.63046 - norm_gini: 0.26956 - norm_gini_val: 0.26092          

Epoch 00003: norm_gini_val improved from 0.24981 to 0.26092, saving model to keras-5fold-run-01-v1-fold-01-run-01.check
Epoch 4/5000
 - 40s - los

Epoch 4/5000
 - 44s - loss: 0.1537 - acc: 0.9636 - val_loss: 0.1526 - val_acc: 0.9635
roc_auc: 0.63735 - roc_auc_val: 0.62901 - norm_gini: 0.27469 - norm_gini_val: 0.25802          

Epoch 00004: norm_gini_val improved from 0.25757 to 0.25802, saving model to keras-5fold-run-01-v1-fold-01-run-02.check
Epoch 5/5000
 - 45s - loss: 0.1533 - acc: 0.9636 - val_loss: 0.1526 - val_acc: 0.9635
roc_auc: 0.64078 - roc_auc_val: 0.63163 - norm_gini: 0.28156 - norm_gini_val: 0.26327          

Epoch 00005: norm_gini_val improved from 0.25802 to 0.26327, saving model to keras-5fold-run-01-v1-fold-01-run-02.check
Epoch 6/5000
 - 45s - loss: 0.1532 - acc: 0.9636 - val_loss: 0.1525 - val_acc: 0.9635
roc_auc: 0.64179 - roc_auc_val: 0.63333 - norm_gini: 0.28359 - norm_gini_val: 0.26666          

Epoch 00006: norm_gini_val improved from 0.26327 to 0.26666, saving model to keras-5fold-run-01-v1-fold-01-run-02.check
Epoch 7/5000
 - 46s - loss: 0.1528 - acc: 0.9636 - val_loss: 0.1523 - val_acc: 0.9635
roc_a

Epoch 7/5000
 - 48s - loss: 0.1529 - acc: 0.9636 - val_loss: 0.1525 - val_acc: 0.9635
roc_auc: 0.64551 - roc_auc_val: 0.63121 - norm_gini: 0.29102 - norm_gini_val: 0.26243          

Epoch 00007: norm_gini_val improved from 0.26036 to 0.26243, saving model to keras-5fold-run-01-v1-fold-02-run-01.check
Epoch 8/5000
 - 49s - loss: 0.1528 - acc: 0.9636 - val_loss: 0.1525 - val_acc: 0.9635
roc_auc: 0.64751 - roc_auc_val: 0.63138 - norm_gini: 0.29502 - norm_gini_val: 0.26277          

Epoch 00008: norm_gini_val improved from 0.26243 to 0.26277, saving model to keras-5fold-run-01-v1-fold-02-run-01.check
Epoch 9/5000
 - 49s - loss: 0.1527 - acc: 0.9636 - val_loss: 0.1527 - val_acc: 0.9635
roc_auc: 0.64888 - roc_auc_val: 0.62832 - norm_gini: 0.29775 - norm_gini_val: 0.25663          

Epoch 00009: norm_gini_val did not improve from 0.26277
Epoch 10/5000
 - 49s - loss: 0.1525 - acc: 0.9636 - val_loss: 0.1525 - val_acc: 0.9635
roc_auc: 0.65086 - roc_auc_val: 0.63102 - norm_gini: 0.30173 - norm_

roc_auc: 0.65325 - roc_auc_val: 0.62965 - norm_gini: 0.30651 - norm_gini_val: 0.25931          

Epoch 00014: norm_gini_val did not improve from 0.26084
Epoch 15/5000
 - 54s - loss: 0.1521 - acc: 0.9636 - val_loss: 0.1524 - val_acc: 0.9635
roc_auc: 0.65415 - roc_auc_val: 0.63092 - norm_gini: 0.30831 - norm_gini_val: 0.26183          

Epoch 00015: norm_gini_val improved from 0.26084 to 0.26183, saving model to keras-5fold-run-01-v1-fold-02-run-02.check
Epoch 16/5000
 - 54s - loss: 0.1521 - acc: 0.9636 - val_loss: 0.1526 - val_acc: 0.9635
roc_auc: 0.65593 - roc_auc_val: 0.63051 - norm_gini: 0.31186 - norm_gini_val: 0.26102          

Epoch 00016: norm_gini_val did not improve from 0.26183
Epoch 17/5000
 - 53s - loss: 0.1520 - acc: 0.9636 - val_loss: 0.1526 - val_acc: 0.9635
roc_auc: 0.65688 - roc_auc_val: 0.62784 - norm_gini: 0.31376 - norm_gini_val: 0.25568          

Epoch 00017: norm_gini_val did not improve from 0.26183
Epoch 18/5000
 - 51s - loss: 0.1520 - acc: 0.9636 - val_loss: 0

 - 60s - loss: 0.1521 - acc: 0.9636 - val_loss: 0.1524 - val_acc: 0.9636
roc_auc: 0.65659 - roc_auc_val: 0.634 - norm_gini: 0.31319 - norm_gini_val: 0.268          

Epoch 00019: norm_gini_val did not improve from 0.26903
Epoch 20/5000
 - 60s - loss: 0.1521 - acc: 0.9636 - val_loss: 0.1524 - val_acc: 0.9636
roc_auc: 0.65642 - roc_auc_val: 0.63335 - norm_gini: 0.31284 - norm_gini_val: 0.2667          

Epoch 00020: norm_gini_val did not improve from 0.26903
Epoch 21/5000
 - 60s - loss: 0.1519 - acc: 0.9636 - val_loss: 0.1524 - val_acc: 0.9636
roc_auc: 0.65707 - roc_auc_val: 0.6335 - norm_gini: 0.31414 - norm_gini_val: 0.267          

Epoch 00021: norm_gini_val did not improve from 0.26903
Epoch 22/5000
 - 60s - loss: 0.1519 - acc: 0.9636 - val_loss: 0.1524 - val_acc: 0.9636
roc_auc: 0.65763 - roc_auc_val: 0.63374 - norm_gini: 0.31527 - norm_gini_val: 0.26749          

Epoch 00022: norm_gini_val did not improve from 0.26903
Epoch 23/5000
 - 60s - loss: 0.1519 - acc: 0.9636 - val_loss: 

 - 65s - loss: 0.1518 - acc: 0.9636 - val_loss: 0.1524 - val_acc: 0.9636
roc_auc: 0.66013 - roc_auc_val: 0.63298 - norm_gini: 0.32026 - norm_gini_val: 0.26597          

Epoch 00023: norm_gini_val did not improve from 0.26891
Epoch 24/5000
 - 65s - loss: 0.1518 - acc: 0.9636 - val_loss: 0.1525 - val_acc: 0.9636
roc_auc: 0.66021 - roc_auc_val: 0.63123 - norm_gini: 0.32042 - norm_gini_val: 0.26246          

Epoch 00024: norm_gini_val did not improve from 0.26891
Epoch 25/5000
 - 65s - loss: 0.1517 - acc: 0.9636 - val_loss: 0.1526 - val_acc: 0.9636
roc_auc: 0.66245 - roc_auc_val: 0.63071 - norm_gini: 0.32491 - norm_gini_val: 0.26142          

Epoch 00025: norm_gini_val did not improve from 0.26891
Epoch 26/5000
 - 65s - loss: 0.1516 - acc: 0.9636 - val_loss: 0.1524 - val_acc: 0.9636
roc_auc: 0.66258 - roc_auc_val: 0.63323 - norm_gini: 0.32516 - norm_gini_val: 0.26645          

Epoch 00026: norm_gini_val did not improve from 0.26891
Epoch 27/5000
 - 65s - loss: 0.1516 - acc: 0.9636 - va

roc_auc: 0.66029 - roc_auc_val: 0.6318 - norm_gini: 0.32057 - norm_gini_val: 0.2636          

Epoch 00026: norm_gini_val did not improve from 0.26556
Epoch 27/5000
 - 71s - loss: 0.1517 - acc: 0.9636 - val_loss: 0.1524 - val_acc: 0.9636
roc_auc: 0.66051 - roc_auc_val: 0.6309 - norm_gini: 0.32102 - norm_gini_val: 0.26179          

Epoch 00027: norm_gini_val did not improve from 0.26556
Epoch 28/5000
 - 71s - loss: 0.1516 - acc: 0.9636 - val_loss: 0.1527 - val_acc: 0.9636
roc_auc: 0.66041 - roc_auc_val: 0.6301 - norm_gini: 0.32083 - norm_gini_val: 0.26021          

Epoch 00028: norm_gini_val did not improve from 0.26556
Epoch 29/5000
 - 70s - loss: 0.1517 - acc: 0.9636 - val_loss: 0.1526 - val_acc: 0.9636
roc_auc: 0.66124 - roc_auc_val: 0.6298 - norm_gini: 0.32248 - norm_gini_val: 0.25959          

Epoch 00029: norm_gini_val did not improve from 0.26556
Epoch 30/5000
 - 71s - loss: 0.1516 - acc: 0.9636 - val_loss: 0.1525 - val_acc: 0.9635
roc_auc: 0.66319 - roc_auc_val: 0.63018 - nor

 - 78s - loss: 0.1518 - acc: 0.9636 - val_loss: 0.1526 - val_acc: 0.9635
roc_auc: 0.66062 - roc_auc_val: 0.6303 - norm_gini: 0.32125 - norm_gini_val: 0.2606          

Epoch 00025: norm_gini_val did not improve from 0.26303
Epoch 26/5000
 - 79s - loss: 0.1516 - acc: 0.9636 - val_loss: 0.1526 - val_acc: 0.9636
roc_auc: 0.66086 - roc_auc_val: 0.62928 - norm_gini: 0.32172 - norm_gini_val: 0.25856          

Epoch 00026: norm_gini_val did not improve from 0.26303
Epoch 00026: early stopping

 Fold 4 Run 2 Log-loss: 0.15241
 Fold 4 Run 2 AUC: 0.63152
 Fold 4 Run 2 normalized gini: 0.26303

 Fold 4 Log-loss: 0.15238
 Fold 4 AUC: 0.63325
 Fold 4 normalized gini: 0.26651

 Time taken: 1 hours 59 minutes and 40.56 seconds.


Here we average all the predictions and provide the final summary.

In [17]:
LL_oof = log_loss(avreal, avpred)
print('\n Average Log-loss: %.5f' % (cv_LL/folds))
print(' Out-of-fold Log-loss: %.5f' % LL_oof)
AUC_oof = roc_auc_score(avreal, avpred)
print('\n Average AUC: %.5f' % (cv_AUC/folds))
print(' Out-of-fold AUC: %.5f' % AUC_oof)
print('\n Average normalized gini: %.5f' % (cv_gini/folds))
print(' Out-of-fold normalized gini: %.5f' % (AUC_oof*2-1))
score = str(round((AUC_oof*2-1), 5))
timer(starttime)
mpred = pred / folds


 Average Log-loss: 0.34292
 Out-of-fold Log-loss: 0.15224

 Average AUC: 1.42437
 Out-of-fold AUC: 0.63438

 Average normalized gini: 0.59874
 Out-of-fold normalized gini: 0.26876

 Time taken: 5 hours 56 minutes and 53.39 seconds.


Save the file with out-of-fold predictions. For easier book-keeping, file names have the out-of-fold gini score and are are tagged by date and time.

In [18]:
print('#\n Writing results')
now = datetime.now()
oof_result = pd.DataFrame(avreal, columns=['target'])
oof_result['prediction'] = avpred
oof_result['id'] = avids
oof_result.sort_values('id', ascending=True, inplace=True)
oof_result = oof_result.set_index('id')
sub_file = 'train_5fold-keras-run-01-v1-oof_' + str(score) + '_' + str(now.strftime('%Y-%m-%d-%H-%M')) + '.csv'
print('\n Writing out-of-fold file:  %s' % sub_file)
oof_result.to_csv(sub_file, index=True, index_label='id')

#
 Writing results

 Writing out-of-fold file:  train_5fold-keras-run-01-v1-oof_0.26876_2019-05-30-01-02.csv


Save the final prediction. This is the one to submit.

In [19]:
result = pd.DataFrame(mpred, columns=['target'])
result['id'] = te_ids
result = result.set_index('id')
print('\n First 10 lines of your 5-fold average prediction:\n')
print(result.head(10))
sub_file = 'submission_5fold-average-keras-run-01-v1_' + str(score) + '_' + str(now.strftime('%Y-%m-%d-%H-%M')) + '.csv'
print('\n Writing submission:  %s' % sub_file)
result.to_csv(sub_file, index=True, index_label='id')


 First 10 lines of your 5-fold average prediction:

      target
id          
0   0.034712
1   0.026840
2   0.026046
3   0.016958
4   0.034355
5   0.042838
6   0.017382
8   0.026070
10  0.057547
11  0.036455

 Writing submission:  submission_5fold-average-keras-run-01-v1_0.26876_2019-05-30-01-02.csv


![](./porto/submit.png)